Quick Drawing is a kaggle challenge for classifying the sketch pictures. The total number of category is 340 and 
there are 50M pictures in the dataset. Each catogory has 25,000 pictures. The difficulty is the drawing might be 
incomplete or the drawing does not match the label. We are trying to develop a CNN to solve the issue.   

According to the challenge, there are 4 kinds of dataset. 

    - Raw data (.ndjson)
    - Simplified drawings (.ndjson)
    - Binary data (.bin)
    - Bitmap data (.npy)
Raw data and Simplified drawings are provided in a vector information of strokes. Bitmap data is rendered into 
28 * 28 grayscale bitmap in the numpy format.  

Maybe we could consider the raw data to improve the performance since it inlcudes the sequence information.


In [1]:
import torch
import time
import torch
import os
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
import numpy as np
from torch.utils import data
import QDdata as QD
import matplotlib.pyplot as plt
%matplotlib inline
from torch.utils.data import DataLoader
import resnet as RN
from torchvision import transforms
import csv

In [2]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 75

print('using device:', device)


using device: cuda


# Data Augmentation

In [3]:
trans = transforms.Compose([transforms.RandomHorizontalFlip(0.5), transforms.ToTensor()])

# Center Feature Per Class

In [4]:
alpha = 0.01
cf_class = torch.from_numpy(np.load("center_feature.npy"))
cf_class = cf_class.to(device=device, dtype=torch.float32)

In [5]:
args = {"learning_rate" : 3e-3}

In [6]:
# The following code comes from https://github.com/pytorch/examples/blob/master/imagenet/main.py
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def adjust_learning_rate(optimizer, epoch, args):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args['learning_rate'] * (0.5 ** epoch )
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

# Resnet50 96X96

In [ ]:
###### The following codes comes from cs231N.
from average_precision import mapk
def mean_ap_torch(y_val,y_pred_score,k):
    y_pred = [torch.sort(c,descending=True)[::-1][1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def mean_ap_3(y_val,y_pred_score,k):
    y_pred = [np.argsort(c)[::-1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def check_accuracy(loader, model):
    
    model.eval()  # set model to evaluation mode
    
    meanAP = []
    meanAP3 = []
    with torch.no_grad():
        for i ,(x,stroke, y) in enumerate(loader):
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            y_pred,_ = model(x)
            y_val = y.reshape(-1,1)
            y_pred = y_pred.data.cpu().numpy()
            
            mAP = 100 * mean_ap_3(y_val,y_pred,1)
            mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
            meanAP.append(mAP)
            meanAP3.append(mAP3)
            if i > 100:
                break
        
        m1 = np.mean(meanAP)
        m3 = np.mean(meanAP3)
        

        if (m3 > QDtrain["mAP3"] + 1) and (m3 > 20):
            if os.path.exists(QDtrain["file"]):
                os.remove(QDtrain["file"])
            QDtrain["file"] = str("model" + QDtrain["prefix"] + str(m3))
            torch.save(model.state_dict(),QDtrain["file"])
            QDtrain["mAP3"] = m3
            QDtrain["mAP"] = m1
        print('Got correct (%.2f,%.2f)' % (m1,m3 ))
        
        
def train(model, optimizer, epochs,args):
    """
    Train a model on quickdrawing.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: A Python integer giving the number of epochs to train for
    - args: argumetns for learning rate
    
    Returns: Nothing, but prints model accuracies during training.
    """

    model = model.to(device=device)  # move the model parameters to CPU/GPU
    loader_val = QD.QDloadStrokeData(val = True)
    loader_val = DataLoader(dataset=loader_val, batch_size=256,shuffle=False)
    
    # We try to aggregate the several batches together 
    # so that we could have a big batchsize to fill in GPU.
    # real_batch size = aggregated_batches * batch_size
    aggregated_batches = 1
    
    for e in range(epochs):

        train_map1 = AverageMeter()
        train_map3 = AverageMeter()
        
        #Learning rate decay
        adjust_learning_rate(optimizer, e, args)
        
        # We split the whole train dataset into 100 segments.
        for i in range(100):
            t1 = time.time()
            total_loss = 0
            train_dataset = QD.QDloadStrokeData(no=i,val = False,transforms = trans)          
            train_loader = DataLoader(dataset=train_dataset, batch_size=256,shuffle=False)
            for t, (x,stroke, y) in enumerate(train_loader):
                model.train()  # put model to training mode
                x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
                y = y.to(device=device, dtype=torch.long)
                #print(stroke.shape)
                #add the center feature returned from resnet 
                scores,cf_pred = model(x)
                
                #Caculate entropy loss
                entropy_loss = F.cross_entropy(scores, y)
                
                #Caculate the center loss 
                center_loss = F.mse_loss(cf_pred,cf_class[y])
                
                loss = entropy_loss + alpha * center_loss
                
                total_loss += loss
                #Calculate train accuracy
                y_pred = scores.data.cpu().numpy()
                y_val = y.reshape(-1,1)
                
                mAP = 100 * mean_ap_3(y_val,y_pred,1)
                mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
             
                train_map1.update(mAP,x.size(0))
                train_map3.update(mAP3,x.size(0))         
                #acc1,acc3 = accuracy(scores, y, topk=(1, 3))
                #train_map1.update(acc1,x.size(0))
                #train_map3.update(acc3,x.size(0))
                                  
                if t % aggregated_batches == 0:                    
                    avg_loss = total_loss / aggregated_batches

                    optimizer.zero_grad()
                    avg_loss.backward()
                    optimizer.step()
                    total_loss = 0

                    t2 = time.time()

                    if t % (print_every) == 0:
                        print('Epoch %d,Iteration %d,loss = %.4f,time %.4f,train accuracy(%.2f,%.2f)' % 
                              (e,t, avg_loss.item(),t2-t1,train_map1.avg,train_map3.avg))
                        check_accuracy(loader_val, model)
                        t1 = time.time()
                        

In [ ]:
cnn_model = RN.resnet50(num_classes=40)
optimizer = optim.Adam(cnn_model.parameters(),lr = args['learning_rate'])
QDtrain = {"mAP":0,"mAP3":0,"file":"","prefix":"resnet50"}

In [ ]:
train(cnn_model,optimizer, epochs=5,args=args)

1. batch size = 64. count =  340 * 10240.
   Iteration 29400, loss = 0.5610
   Got 64 correct (80.22,85.62)
2. batch size =128 count = 340 *10240 * 2
    Iteration 37800, loss = 0.9535
    Got 128 correct (77.65,83.64)
3. batch size =128 count = 340 *10240 * 2 
    train on 28 * 28 drawing transfered from stroke
    Iteration 45900, loss = 0.9149
    Got correct (57.55,65.98)

## RNN Model

In [10]:
import torch.nn.functional as F
class SketchRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout = 0.1, n_layers=1):
        super(SketchRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.conv1d_1 = nn.Conv1d(input_size, 48, 5)
        self.dropout_1 = nn.Dropout(0.1)
        self.conv1d_2 = nn.Conv1d(48, 64, 5)
        self.dropout_2 = nn.Dropout(0.1)
        self.conv1d_3 = nn.Conv1d(64, 96, 3)
        self.dropout_3 = nn.Dropout(0.1)
        self.lstm_1 = nn.LSTM(96,hidden_size, n_layers, dropout,batch_first=True,bidirectional=True)
        self.fc_mu = nn.Linear(hidden_size*186*2, output_size)
        #self.fc_output = nn.Linear(2048, output_size)
        

    def forward(self, inputs, hidden):
        inputs = inputs.transpose(1, 2)
        
        output = self.conv1d_1(inputs)
        output = self.dropout_1(output)
        output = self.conv1d_2(output)
        output = self.dropout_2(output)
        output = self.conv1d_3(output)
        output = self.dropout_3(output)
        output = output.transpose(1, 2)
        
        output, (hidden,x) = self.lstm_1(output, hidden)
        
        output = output.contiguous()
        output = output.view(output.size(0),-1)
        output = self.fc_mu(output)
        #output = self.fc_output(output)
        output = F.log_softmax(output, dim=1)
        return output, hidden 

rnn_model = SketchRNN(3, 256, 340,dropout=0.1).to(device)
optimizer = optim.Adam(rnn_model.parameters(),weight_decay = 1e-5,lr=0.1)

In [11]:
# The following codes comes from cs231N.
from average_precision import mapk
def mean_ap_torch(y_val,y_pred_score,k):
    y_pred = [torch.sort(c,descending=True)[::-1][1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def mean_ap_3(y_val,y_pred_score,k):
    y_pred = [np.argsort(c)[::-1][:k] for c in y_pred_score]
    return mapk(y_val,y_pred,k)

def check_accuracy(loader, model):
    
    model.eval()  # set model to evaluation mode
    
    meanAP = []
    meanAP3 = []
    with torch.no_grad():
        for i ,(_,x, y) in enumerate(loader):
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            y_pred,_ = model(x,None)
            y_val = y.reshape(-1,1)
            y_pred = y_pred.data.cpu().numpy()
            
            mAP = 100 * mean_ap_3(y_val,y_pred,1)
            mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
            meanAP.append(mAP)
            meanAP3.append(mAP3)
            if i > 100:
                break
        
        m1 = np.mean(meanAP)
        m3 = np.mean(meanAP3)
        

        if (m3 > QDtrain["mAP3"] + 1) and (m3 > 20):
            if os.path.exists(QDtrain["file"]):
                os.remove(QDtrain["file"])
            QDtrain["file"] = str("model" + QDtrain["prefix"] + str(m3))
            torch.save(model.state_dict(),QDtrain["file"])
            QDtrain["mAP3"] = m3
            QDtrain["mAP"] = m1
        print('Got correct (%.2f,%.2f)' % (m1,m3 ))
        
        
def train(model, optimizer, epochs,args):
    """
    Train a model on quickdrawing.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: A Python integer giving the number of epochs to train for
    - args: argumetns for learning rate
    
    Returns: Nothing, but prints model accuracies during training.
    """

    model = model.to(device=device)  # move the model parameters to CPU/GPU
    loader_val = QD.QDloadStrokeData(val = True)
    loader_val = DataLoader(dataset=loader_val, batch_size=512,shuffle=False)
    
    # We try to aggregate the several batches together 
    # so that we could have a big batchsize to fill in GPU.
    # real_batch size = aggregated_batches * batch_size
    aggregated_batches = 1
    
    for e in range(epochs):

        train_map1 = AverageMeter()
        train_map3 = AverageMeter()
        
        #Learning rate decay
        adjust_learning_rate(optimizer, e, args)
        
        # We split the whole train dataset into 100 segments.
        for i in range(100):
            t1 = time.time()
            total_loss = 0
            train_dataset = QD.QDloadStrokeData(no=i,val = False,transforms = trans)          
            train_loader = DataLoader(dataset=train_dataset, batch_size=512,shuffle=False)
            for t, (_, x, y) in enumerate(train_loader):
                model.train()  # put model to training mode
                x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
                y = y.to(device=device, dtype=torch.long)
                
                #add the center feature returned from resnet 
                scores,_ = model(x,None)
                #Caculate entropy loss
                entropy_loss = F.cross_entropy(scores, y)
                
                total_loss += entropy_loss
                #Calculate train accuracy
                y_pred = scores.data.cpu().numpy()
                y_val = y.reshape(-1,1)
                
                mAP = 100 * mean_ap_3(y_val,y_pred,1)
                mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
             
                train_map1.update(mAP,x.size(0))
                train_map3.update(mAP3,x.size(0))         
                #acc1,acc3 = accuracy(scores, y, topk=(1, 3))
                #train_map1.update(acc1,x.size(0))
                #train_map3.update(acc3,x.size(0))
                                  
                if t % aggregated_batches == 0:                    
                    avg_loss = total_loss / aggregated_batches

                    optimizer.zero_grad()
                    avg_loss.backward()
                    optimizer.step()
                    total_loss = 0

                    t2 = time.time()

                    if t % (print_every) == 0:
                        print('Epoch %d,Iteration %d,loss = %.4f,time %.4f,train accuracy(%.2f,%.2f)' % 
                              (e,t, avg_loss.item(),t2-t1,train_map1.avg,train_map3.avg))
                        check_accuracy(loader_val, model)
                        t1 = time.time()
                        

In [12]:
QDtrain = {"mAP":0,"mAP3":0,"file":"","prefix":"resnet50"}
args = {"learning_rate" : 3e-4}
optimizer = optim.Adam(rnn_model.parameters(),lr = args['learning_rate'])
train(rnn_model,optimizer, epochs=5,args=args)


No = 0 and total number of items 20480
No = 0 and total number of items 42546
Epoch 0,Iteration 0,loss = 5.8329,time 17.8096,train accuracy(0.78,1.01)
Got correct (9.17,14.98)
Epoch 0,Iteration 75,loss = 2.2627,time 18.6264,train accuracy(26.78,36.23)
Got correct (39.47,50.21)
No = 1 and total number of items 42279
Epoch 0,Iteration 0,loss = 2.1763,time 18.2252,train accuracy(28.00,37.61)
Got correct (38.58,49.44)
Epoch 0,Iteration 75,loss = 1.8551,time 18.6740,train accuracy(34.79,45.14)
Got correct (47.51,58.59)
No = 2 and total number of items 42425
Epoch 0,Iteration 0,loss = 1.8765,time 18.3277,train accuracy(35.36,45.73)
Got correct (47.50,58.48)
Epoch 0,Iteration 75,loss = 1.6040,time 18.3241,train accuracy(39.66,50.18)
Got correct (52.73,63.35)
No = 3 and total number of items 42530
Epoch 0,Iteration 0,loss = 1.6343,time 17.9596,train accuracy(40.08,50.58)
Got correct (53.84,64.45)
Epoch 0,Iteration 75,loss = 1.5478,time 18.5729,train accuracy(43.15,53.69)
Got correct (56.16,66.

Got correct (74.68,82.63)
No = 34 and total number of items 42373
Epoch 0,Iteration 0,loss = 0.9604,time 17.6264,train accuracy(64.23,73.74)
Got correct (74.56,82.48)
Epoch 0,Iteration 75,loss = 0.8918,time 18.4531,train accuracy(64.45,73.93)
Got correct (75.04,83.02)
No = 35 and total number of items 42258
Epoch 0,Iteration 0,loss = 0.9024,time 17.3975,train accuracy(64.47,73.95)
Got correct (74.90,82.92)
Epoch 0,Iteration 75,loss = 0.8076,time 18.6546,train accuracy(64.68,74.14)
Got correct (75.03,82.99)
No = 36 and total number of items 42282
Epoch 0,Iteration 0,loss = 0.9391,time 17.3863,train accuracy(64.70,74.16)
Got correct (75.07,83.02)
Epoch 0,Iteration 75,loss = 0.9364,time 18.6848,train accuracy(64.91,74.34)
Got correct (75.27,83.08)
No = 37 and total number of items 42337
Epoch 0,Iteration 0,loss = 0.8359,time 17.5202,train accuracy(64.93,74.36)
Got correct (75.37,83.19)
Epoch 0,Iteration 75,loss = 0.7929,time 18.7264,train accuracy(65.13,74.54)
Got correct (75.51,83.37)
No

Got correct (79.02,86.15)
No = 68 and total number of items 42187
Epoch 0,Iteration 0,loss = 0.7492,time 17.5343,train accuracy(69.84,78.58)
Got correct (78.77,85.99)
Epoch 0,Iteration 75,loss = 0.6879,time 18.6584,train accuracy(69.94,78.66)
Got correct (79.28,86.34)
No = 69 and total number of items 42483
Epoch 0,Iteration 0,loss = 0.6458,time 17.6815,train accuracy(69.95,78.67)
Got correct (79.11,86.30)
Epoch 0,Iteration 75,loss = 0.6939,time 18.6493,train accuracy(70.05,78.75)
Got correct (79.66,86.69)
No = 70 and total number of items 42641
Epoch 0,Iteration 0,loss = 0.7822,time 17.5813,train accuracy(70.06,78.76)
Got correct (79.63,86.59)
Epoch 0,Iteration 75,loss = 0.7192,time 18.6623,train accuracy(70.15,78.84)
Got correct (79.41,86.47)
No = 71 and total number of items 42289
Epoch 0,Iteration 0,loss = 0.6692,time 17.6640,train accuracy(70.16,78.85)
Got correct (79.38,86.45)
Epoch 0,Iteration 75,loss = 0.7243,time 18.5494,train accuracy(70.26,78.93)
Got correct (79.91,86.84)
No

Got correct (82.28,88.68)
No = 2 and total number of items 42425
Epoch 1,Iteration 0,loss = 0.6198,time 17.7883,train accuracy(80.30,87.18)
Got correct (82.38,88.74)
Epoch 1,Iteration 75,loss = 0.5790,time 18.5121,train accuracy(80.55,87.36)
Got correct (82.34,88.73)
No = 3 and total number of items 42530
Epoch 1,Iteration 0,loss = 0.5340,time 17.7580,train accuracy(80.55,87.37)
Got correct (82.37,88.74)
Epoch 1,Iteration 75,loss = 0.6102,time 18.6894,train accuracy(80.63,87.42)
Got correct (82.35,88.66)
No = 4 and total number of items 42463
Epoch 1,Iteration 0,loss = 0.5384,time 17.8781,train accuracy(80.63,87.42)
Got correct (82.43,88.75)
Epoch 1,Iteration 75,loss = 0.5518,time 18.6751,train accuracy(80.63,87.43)
Got correct (82.13,88.55)
No = 5 and total number of items 41893
Epoch 1,Iteration 0,loss = 0.6099,time 17.4629,train accuracy(80.64,87.44)
Got correct (82.45,88.75)
Epoch 1,Iteration 75,loss = 0.5721,time 19.0453,train accuracy(80.73,87.50)
Got correct (82.44,88.79)
No = 6

Got correct (83.24,89.34)
No = 36 and total number of items 42282
Epoch 1,Iteration 0,loss = 0.5391,time 17.9624,train accuracy(81.50,88.10)
Got correct (83.28,89.38)
Epoch 1,Iteration 75,loss = 0.5248,time 18.5748,train accuracy(81.52,88.11)
Got correct (83.26,89.34)
No = 37 and total number of items 42337
Epoch 1,Iteration 0,loss = 0.5849,time 17.9024,train accuracy(81.53,88.12)
Got correct (83.13,89.28)
Epoch 1,Iteration 75,loss = 0.5039,time 18.5456,train accuracy(81.54,88.13)
Got correct (83.23,89.36)
No = 38 and total number of items 42490
Epoch 1,Iteration 0,loss = 0.5837,time 17.8366,train accuracy(81.54,88.13)
Got correct (83.06,89.21)
Epoch 1,Iteration 75,loss = 0.4811,time 18.7300,train accuracy(81.57,88.15)
Got correct (83.22,89.29)
No = 39 and total number of items 42296
Epoch 1,Iteration 0,loss = 0.6027,time 17.8789,train accuracy(81.58,88.15)
Got correct (82.98,89.20)
Epoch 1,Iteration 75,loss = 0.5031,time 18.8506,train accuracy(81.59,88.16)
Got correct (83.33,89.35)
No

...

Got correct (85.74,91.16)
No = 40 and total number of items 42336
Epoch 3,Iteration 0,loss = 0.3690,time 18.1481,train accuracy(85.65,91.13)
Got correct (85.69,91.14)
Epoch 3,Iteration 75,loss = 0.4092,time 18.8729,train accuracy(85.66,91.13)
Got correct (85.55,91.04)
No = 41 and total number of items 42321
Epoch 3,Iteration 0,loss = 0.4566,time 18.1820,train accuracy(85.66,91.13)
Got correct (85.65,91.09)
Epoch 3,Iteration 75,loss = 0.4041,time 19.0462,train accuracy(85.67,91.13)
Got correct (85.80,91.23)
No = 42 and total number of items 42427
Epoch 3,Iteration 0,loss = 0.3980,time 17.8841,train accuracy(85.67,91.13)
Got correct (85.70,91.18)
Epoch 3,Iteration 75,loss = 0.5059,time 18.9088,train accuracy(85.68,91.14)
Got correct (85.72,91.17)
No = 43 and total number of items 42466
Epoch 3,Iteration 0,loss = 0.3875,time 17.9315,train accuracy(85.68,91.14)
Got correct (85.69,91.17)
Epoch 3,Iteration 75,loss = 0.4750,time 18.8567,train accuracy(85.68,91.14)
Got correct (85.73,91.14)
No

Got correct (85.90,91.27)
No = 74 and total number of items 42333
Epoch 3,Iteration 0,loss = 0.4717,time 18.1248,train accuracy(85.91,91.30)
Got correct (85.70,91.17)
Epoch 3,Iteration 75,loss = 0.4188,time 18.8470,train accuracy(85.92,91.31)
Got correct (85.80,91.22)
No = 75 and total number of items 42454
Epoch 3,Iteration 0,loss = 0.3576,time 17.7249,train accuracy(85.92,91.31)
Got correct (85.81,91.21)
Epoch 3,Iteration 75,loss = 0.3851,time 18.9474,train accuracy(85.93,91.31)
Got correct (85.91,91.26)
No = 76 and total number of items 42653
Epoch 3,Iteration 0,loss = 0.4783,time 18.2625,train accuracy(85.93,91.31)
Got correct (85.91,91.28)
Epoch 3,Iteration 75,loss = 0.3768,time 18.7944,train accuracy(85.93,91.32)
Got correct (85.64,91.13)
No = 77 and total number of items 42327
Epoch 3,Iteration 0,loss = 0.4329,time 17.9532,train accuracy(85.94,91.32)
Got correct (85.61,91.10)
Epoch 3,Iteration 75,loss = 0.3884,time 18.6467,train accuracy(85.94,91.32)
Got correct (85.83,91.23)
No

Got correct (85.80,91.26)
No = 8 and total number of items 42494
Epoch 4,Iteration 0,loss = 0.4126,time 18.0156,train accuracy(86.08,91.43)
Got correct (85.91,91.30)
Epoch 4,Iteration 75,loss = 0.4302,time 18.7841,train accuracy(86.07,91.42)
Got correct (85.71,91.21)
No = 9 and total number of items 42535
Epoch 4,Iteration 0,loss = 0.3971,time 18.0956,train accuracy(86.07,91.42)
Got correct (85.73,91.22)
Epoch 4,Iteration 75,loss = 0.3511,time 18.5665,train accuracy(86.08,91.43)
Got correct (86.00,91.36)
No = 10 and total number of items 42232
Epoch 4,Iteration 0,loss = 0.3984,time 17.9571,train accuracy(86.09,91.43)
Got correct (85.90,91.32)
Epoch 4,Iteration 75,loss = 0.4250,time 18.5478,train accuracy(86.09,91.44)
Got correct (85.96,91.34)
No = 11 and total number of items 42595
Epoch 4,Iteration 0,loss = 0.4120,time 17.7353,train accuracy(86.10,91.44)
Got correct (85.96,91.35)
Epoch 4,Iteration 75,loss = 0.4054,time 18.8116,train accuracy(86.11,91.45)
Got correct (85.88,91.31)
No =

Got correct (86.03,91.37)
No = 42 and total number of items 42427
Epoch 4,Iteration 0,loss = 0.3770,time 17.8561,train accuracy(86.31,91.58)
Got correct (86.03,91.36)
Epoch 4,Iteration 75,loss = 0.4469,time 18.9673,train accuracy(86.31,91.58)
Got correct (86.06,91.39)
No = 43 and total number of items 42466
Epoch 4,Iteration 0,loss = 0.4069,time 18.1384,train accuracy(86.31,91.58)
Got correct (85.97,91.34)
Epoch 4,Iteration 75,loss = 0.4820,time 18.8382,train accuracy(86.32,91.59)
Got correct (86.01,91.37)
No = 44 and total number of items 42053
Epoch 4,Iteration 0,loss = 0.4289,time 18.0195,train accuracy(86.32,91.59)
Got correct (86.06,91.40)
Epoch 4,Iteration 75,loss = 0.3863,time 18.9416,train accuracy(86.32,91.59)
Got correct (86.10,91.43)
No = 45 and total number of items 42254
Epoch 4,Iteration 0,loss = 0.3681,time 18.1376,train accuracy(86.32,91.59)
Got correct (86.04,91.37)
Epoch 4,Iteration 75,loss = 0.4535,time 18.7107,train accuracy(86.33,91.59)
Got correct (86.00,91.37)
No

Got correct (86.10,91.45)
No = 76 and total number of items 42653
Epoch 4,Iteration 0,loss = 0.4667,time 18.0769,train accuracy(86.52,91.72)
Got correct (86.15,91.48)
Epoch 4,Iteration 75,loss = 0.3938,time 18.7336,train accuracy(86.52,91.72)
Got correct (86.04,91.40)
No = 77 and total number of items 42327
Epoch 4,Iteration 0,loss = 0.3633,time 17.9784,train accuracy(86.52,91.72)
Got correct (86.02,91.39)
Epoch 4,Iteration 75,loss = 0.3820,time 18.6509,train accuracy(86.52,91.73)
Got correct (86.00,91.37)
No = 78 and total number of items 42504
Epoch 4,Iteration 0,loss = 0.4241,time 17.9551,train accuracy(86.52,91.73)
Got correct (86.09,91.40)
Epoch 4,Iteration 75,loss = 0.3165,time 18.7284,train accuracy(86.53,91.73)
Got correct (86.11,91.45)
No = 79 and total number of items 42552
Epoch 4,Iteration 0,loss = 0.2964,time 17.7312,train accuracy(86.53,91.73)
Got correct (86.07,91.40)
Epoch 4,Iteration 75,loss = 0.3571,time 18.7079,train accuracy(86.54,91.74)
Got correct (86.13,91.45)
No

# Run test samples

In [ ]:
torch.save(model.state_dict(),"center-loss93.93")

In [ ]:
def evaluate_test(test_data, model):
    model.eval()  
    with torch.no_grad():
        meanAP = []
        meanAP3 = []
        center_feature = []
        
        for i ,(x,y) in enumerate(test_data):
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            y_pred,feature_pred = model(x)
            y_val = y.reshape(-1,1)
            y_pred = y_pred.data.cpu().numpy()
            feature_pred = feature_pred.data.cpu().numpy()
            
            mAP = 100 * mean_ap_3(y_val,y_pred,1)
            mAP3 = 100 * mean_ap_3(y_val,y_pred,3)
            meanAP.append(mAP)
            meanAP3.append(mAP3)

            center_feature.extend(feature_pred)
            
        m1 = np.mean(meanAP)
        m3 = np.mean(meanAP3)
        print(len(center_feature),center_feature[0].shape)
        return (m1,m3,center_feature)

model = model.to(device=device)
checkpoint = torch.load("modelresnet5093.680016")
model.load_state_dict(checkpoint)

class_name = QD.qd_names

class_accuracy = []

center_feature = []

val_accuracy_map1 = AverageMeter()
val_accuracy_map3 = AverageMeter()

for i,name in enumerate(tqdm(class_name)):
    t1 = time.time()
    batch_size = 128
    data_file = os.path.join("./train",name+".csv")
    test_data = QD.QDloadStrokeData(data_file = data_file)
    test_loader = DataLoader(dataset=test_data, batch_size=batch_size, num_workers=2, shuffle=False)

    m1,m3,feature_list = evaluate_test(test_loader,model)
    
    feature_list = np.array(feature_list)
    print(feature_list.shape)
    
    #feature_list is N * 2048
    
    center_feature.append(np.mean(feature_list,axis = 0))

    class_accuracy.append((name,str(m1),str(m3)))
    
    val_accuracy_map1.update(m1,feature_list[0])
    val_accuracy_map3.update(m3,feature_list[0])
    
    print(name,str(m1),str(m3))
    t2 = time.time()
    print("Time:",t2-t1)
    
with open('class_accuracy.csv',"w+") as fp:
    writer = csv.writer(fp, delimiter=',')
    writer.writerow(["class", "meanAP1", "meanAP3"])  # write header
    writer.writerows(class_accuracy)

print("The average accuracy %.4f %.4f",val_accuracy_map1.avg,val_accuracy_map3.avg)

np.save("center_feature",center_feature)



1) 96*96, resnet50, epoch 10, GSD. 75.2%
2) 96*96, resnet50, epoch 10, GSD  79.9% model_parametertensor(91.6667)
3) 96*96, resnet50, epoch 10, Adam model_resnet_92.317_90


* convert simplified test dataset to pixel drawsing

Check the accuracy based on the class.

Create test_simplified.
load compressed(96*96) 1.968482494354248
load non compressed(96*96)  0.31299567222595215
load non compressed(28*28) 0.04715704917907715


In [ ]:
for i in range(10,11):
    data = QD.QDcreateData()
    data.create(start=i*3000,dir_name="../cs230/pic96")
    del data

In [ ]:
torch.cuda.empty_cache()